In [8]:
import os
workDir = os.getcwd()
print(workDir)

/Users/Matthew/Documents/SchoolWork/physics/GradNotes/ParticlePhysics/PostdocResearch/HGC/EstToolsSUSY/HGCStudies


In [232]:
import os
import io
import pandas as pd
import numpy as np

sensorfilename = workDir + '/Fiducials/sensorfiducialtrialrun.xls'
fiducialfilename = workDir + '/Fiducials/GantryTrayFiducialData.xls'

dfs = pd.read_excel(sensorfilename)
dff = pd.read_excel(fiducialfilename)
dfs = dfs.dropna(how='all')
dff = dff.dropna(how='all')

In [233]:
X = dfs.index[dfs['Unnamed: 3'] == 'X'].tolist()
sPoints = []
for x in X:
    if 'Fiducial' in dfs['Unnamed: 2'][x]:
        sPoints.append((float(dfs['Unnamed: 4'][x]), float(dfs['Unnamed: 4'][x + 1])))       

# Function to sort hte list by second item of tuple
def Sort_Tuple(tup): 
  
    # reverse = True (Sorts in descending order for Y coordinate)
    # reverse = None (Sorts in ascending order for X coordinate)
    # key is set to sort using first or second element of 
    # sublist lambda has been used 
    tup.sort(key = lambda x: x[1], reverse=True)
    tup.sort(key = lambda x: x[0])
    return tup 

sPoints = Sort_Tuple(sPoints)

In [235]:
P1 = []
P2 = []
trayPoints = []
X = dff.index.tolist()

for x in X:
    if dff['Unnamed: 2'][x] == 'TrayFiducial2Fine2.X':
        trayPoints.append((float(0.), float(0.)))
        trayPoints.append((float(dff['Unnamed: 3'][x]), float(dff['Unnamed: 3'][x + 1])))
    elif 'X' in dff['Unnamed: 2'][x] and 'P1' in dff['Unnamed: 2'][x]:
        P1.append((float(dff['Unnamed: 3'][x]), float(dff['Unnamed: 3'][x + 1])))
    elif 'X' in dff['Unnamed: 2'][x] and 'P2' in dff['Unnamed: 2'][x]:
        P2.append((float(dff['Unnamed: 3'][x]), float(dff['Unnamed: 3'][x + 1])))
        
#fPoints = Sort_Tuple(fPoints)

In [255]:
def distance(tup):
    r = np.sqrt(((tup[1][0] - tup[0][0])**2) + ((tup[1][1] - tup[0][1])**2))
    return r
    
def angleOfFrames(Fiducial, baseplate):
    f = distance(Fiducial)
    b = distance(baseplate)
    th1 = 0.
    if (baseplate[0][0] - baseplate[1][0]) > 0.:
        b1, b2 = (baseplate[0][0] - baseplate[1][0]), (baseplate[0][1] - baseplate[1][1])
    else:
        b1, b2 = (baseplate[1][0] - baseplate[0][0]), (baseplate[1][1] - baseplate[0][1])
        
    th1 = Fiducial[1][0]*b1 + Fiducial[1][1]*b2
    theta = np.pi/2 - np.arccos((th1)/(f*b))
    return theta

def transform(points, theta, origin):
    c, s = np.cos(theta), np.sin(theta)
    rotate = np.array(((c, -s), (s, c)))
    return np.dot((points - origin), rotate.T)

def angleDeviation(coord):
    rad = []
    deg = []
    for x in range(0, int(len(coord)/2)):
        r = np.arctan2((coord[x + 2][0] - coord[x][0]), (coord[x + 2][1] - coord[x][1]))
        rad.append(r)
        deg.append(np.degrees(r))
        
    return rad, deg

In [251]:
angleP1 = angleOfFrames(trayPoints, P1)
angleP2 = angleOfFrames(trayPoints, P2)
transformP1 = transform(np.array(sPoints), angleP1, np.array(P1[0]))
#transform(np.array(sPoints), angleP2, np.array(P2[0]))

In [256]:
radP1, degP1 = angleDeviation(transformP1)
print(radP1)
print(degP1)

IndexError: index 4 is out of bounds for axis 0 with size 4